In [2]:
#import the required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from glob import glob

In [ ]:
tf.__version__

'2.3.0'

In [6]:
#resize all images to the expected size
image_size = [224, 224]

#set train, test, validation dataset path
train_path = '/content/drive/My Drive/Datasets/Pneumonia Chest Xray/train'
valid_path = '/content/drive/My Drive/Datasets/Pneumonia Chest Xray/val'
test_path = '/content/drive/My Drive/Datasets/Pneumonia Chest Xray/test'

In [ ]:
#import the VGG16 architecture and add preprocessing layer, we are using ImageNet weights
VGG16_model = keras.applications.vgg16.VGG16(input_shape = image_size + [3], weights = 'imagenet', include_top = False)

58892288/58889256 [==============================] - 1s 0us/step


In [ ]:
#freeze the weights of the pre-trained layers
for layer in VGG16_model.layers:
  layer.trainable = False

In [ ]:
#useful for getting number of output classes
folders = glob('/content/drive/My Drive/Datasets/Pneumonia Chest Xray/train/*')
folders

['/content/drive/My Drive/Datasets/Pneumonia Chest Xray/train/NORMAL',
 '/content/drive/My Drive/Datasets/Pneumonia Chest Xray/train/PNEUMONIA']

In [ ]:
VGG16_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
#adding our own layers
layer_flatten = keras.layers.Flatten()(VGG16_model.output)
output = keras.layers.Dense(len(folders), activation = "softmax")(layer_flatten)
model = keras.Model(inputs = VGG16_model.input, outputs = output)

In [ ]:
#summary of our model
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [ ]:
#compile the model and specify loss function and optimizer
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [4]:
#use the ImageDataGenerator class to load images from the dataset
train_data_generator = keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

valid_data_generator = keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)

test_data_generator = keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)

In [7]:
#make sure you provide the same target size as initialied for the image size
training_set = train_data_generator.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

validation_set = valid_data_generator.flow_from_directory(valid_path,
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'categorical')

test_set = test_data_generator.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'categorical')

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
#train the model
history = model.fit_generator(
  training_set,
  validation_data = validation_set,
  epochs = 5,
  steps_per_epoch = len(training_set),
  validation_steps = len(validation_set)
)

#save the model as an h5 file
model.save('/content/drive/My Drive/Datasets/Pneumonia Chest Xray/model_vgg16_new.h5')

Epoch 1/5
326/326 [==============================] - 2913s 9s/step - loss: 0.1725 - accuracy: 0.9392 - val_loss: 0.4734 - val_accuracy: 0.7500
Epoch 2/5
326/326 [==============================] - 2690s 8s/step - loss: 0.1363 - accuracy: 0.9530 - val_loss: 0.2746 - val_accuracy: 0.8750
Epoch 3/5
326/326 [==============================] - 2693s 8s/step - loss: 0.1215 - accuracy: 0.9609 - val_loss: 0.5689 - val_accuracy: 0.7500
Epoch 4/5
326/326 [==============================] - 2691s 8s/step - loss: 0.1126 - accuracy: 0.9618 - val_loss: 0.0950 - val_accuracy: 1.0000
Epoch 5/5
326/326 [==============================] - 2687s 8s/step - loss: 0.0803 - accuracy: 0.9693 - val_loss: 0.4396 - val_accuracy: 0.8750


In [ ]:
#plot the loss
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

#plot the accuracy
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [3]:
model = keras.models.load_model('/content/drive/My Drive/Datasets/Pneumonia Chest Xray/model_vgg16.h5')

In [8]:
model.evaluate(test_set)

39/39 [==============================] - 304s 8s/step - loss: 0.4958 - accuracy: 0.8958


[0.49577927589416504, 0.8958333134651184]

In [ ]:
#using the model to make predictions
y_pred = model.predict(test_set)
y_pred = np.argmax(y_pred, axis=1)
y_pred

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,